In [ ]:
#!pip install pero-ocr
#O Kodym, M Hradiš: Page Layout Analysis System for Unconstrained Historic Documents. ICDAR, 2021.
#M Kišš, K Beneš, M Hradiš: AT-ST: Self-Training Adaptation Strategy for OCR in Domains with Limited Transcriptions. ICDAR, 2021.
#J Kohút, M Hradiš: TS-Net: OCR Trained to Switch Between Text Transcription Styles. ICDAR, 2021.

In [1]:
import os
import configparser
import cv2
import numpy as np
from lxml import etree
from pero_ocr.document_ocr.layout import PageLayout
from pero_ocr.document_ocr.page_parser import PageParser
import shutil
import pandas as pd
import re

/home/mailan/anaconda3/envs/Mémoire/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/mailan/anaconda3/envs/Mémoire/lib/python3.9/site-packages/numba/__init__.py", line 42, in <module>
    from numba.np.ufunc import (vectorize, guvectorize, threading_layer,
  File "/home/mailan/anaconda3/envs/Mémoire/lib/python3.9/site-packages/numba/np/ufunc/__init__.py", line 3, in <module>
    from numba.np.ufunc.decorators import Vectorize, GUVectorize, vectorize, guvectorize
  File "/home/mailan/anaconda3/envs/Mémoire/lib/python3.9/site-packages/numba/np/ufunc/decorators.py", line 3, in <module>
    from numba.np.ufunc import _internal
SystemError: initialization of _internal failed without raising an exception


cannot import numba, creating dummy jit definition


In [ ]:
# Code Pero-OCR

path="./Annonces_Lyon"

os.chdir(path)
for subdir,dirs,files in os.walk(path, topdown=True):
    for file in files:
        if file.endswith(".jpg") or file.endswith(".tif"):
            path_file=os.path.join(path, subdir)
            os.chdir(path_file)
            
            # configuration
            config_path = "./pero-ocr-modele/config.ini"
            config = configparser.ConfigParser()
            config.read(config_path)
            page_parser = PageParser(config, config_path=os.path.dirname(config_path))
            input_image_path = os.path.join(path_file, file)
            
            # lecture de l'image
            image = cv2.imread(input_image_path, 1)
            page_layout = PageLayout(id=input_image_path,page_size=(image.shape[0], image.shape[1]))
            
            # océrisation
            page_layout = page_parser.process_page(image, page_layout)
            
            # création d'un nouveau dossier pour stocker les résultats de l'OCR
            path_resultats="./OCR_Annonces"
            os.chdir(path_resultats)
            nom_dossier_resultats="OCR_"+os.path.splitext(os.path.basename(path_file))[0]
            if not os.path.exists(nom_dossier_resultats):
                os.mkdir(nom_dossier_resultats)
            new_path_resultats=os.path.join(path_resultats, nom_dossier_resultats)
            os.chdir(new_path_resultats)
            nom_fichier='OCR_'+os.path.splitext(os.path.basename(file))[0]+'.xml'
            
            # enregistrement du fichier XML
            page_layout.to_pagexml(nom_fichier)
            
            

In [ ]:
# extraire le text brut à partir des xml

path = './OCR_Annonces_Lyon'
os.chdir(path)
for dossier_année in os.listdir(path):
    new_path=os.path.join(path, dossier_année)
    os.chdir(new_path)
    for dossier_date in os.listdir(new_path):
        newer_path=os.path.join(new_path,dossier_date)
        os.chdir(newer_path)
        for fichier in os.scandir(newer_path):
            file_path = os.path.join(new_path, fichier)
            tree = etree.parse(file_path)
            file_name=os.path.splitext(os.path.basename(fichier))[0]+'.txt'
            with open(file_name, 'w') as fichier_texte:
                for elt in tree.iter('{http://schema.primaresearch.org/PAGE/gts/pagecontent/2019-07-15}Unicode'):
                    try: fichier_texte.write(elt.text+'\n')
                    except:
                        pass

In [ ]:
# concaténer tous les .txt par numéro

from natsort import natsorted

path = './OCR_Annonces_Lyon'
os.chdir(path)
for dossier_année in os.listdir(path):
    new_path=os.path.join(path, dossier_année)
    os.chdir(new_path)
    for dossier_date in os.listdir(new_path):
        newer_path=os.path.join(new_path,dossier_date)
        os.chdir(newer_path)

        new_file_name="FULL_"+os.path.basename(dossier_date)+".txt"

        with open(new_file_name, "w") as new_file:
            for name in natsorted(os.listdir(newer_path)):
                if name.endswith(".txt"):
                    with open(name) as f:
                        for line in f:
                            new_file.write(line)
                        new_file.write("\n")

In [ ]:
# extraire les FULL_[...].txt dans un nouveau dossier séparé

path = './OCR_Annonces_Lyon'

for dossier_année in os.listdir(path):
    new_path=os.path.join(path,dossier_année)
    os.chdir(new_path)
    for dossier_date in os.listdir(new_path):
        newer_path=os.path.join(new_path,dossier_date)
        os.chdir(newer_path)
        for fichier in os.listdir(newer_path):
            if fichier.startswith("FULL"):
                final_path=os.path.join(path,"FULL_"+(os.path.basename(dossier_année)))
                if not os.path.exists(final_path):
                    os.mkdir(final_path)
                shutil.move(fichier, final_path)
